# 라이브러리 Install

In [64]:
%pip install --upgrade pip

%pip install qdrant_client rank_bm25 sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


# 라이브러리 Import

In [65]:
from collections import defaultdict
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, SparseVectorParams, PointStruct, SparseVector
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

import numpy as np

# 임베딩 모델, 벡터 DB 초기화

  📊 사용 가능한 로컬 임베딩 모델

  | 모델                                    | 차원  | 속도     | 품질      | 용도                    |
  |---------------------------------------|-----|--------|---------|-----------------------|
  | all-MiniLM-L6-v2                      | 384 | ⚡⚡⚡ 빠름 | ⭐⭐⭐ 좋음  | 빠른 테스트                |
  | all-mpnet-base-v2                     | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 추천 (Vertex AI와 동일 차원) |
  | paraphrase-multilingual-mpnet-base-v2 | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 한국어 지원 좋음             |

In [66]:
# ========================================
# 1. 초기화
# ========================================
model = "all-MiniLM-L6-v2"
sentenceTransformer = SentenceTransformer(model)
print(f"✅ SentenceTransformer 모델 초기화 완료 ({model})")

qdrantClient = QdrantClient(":memory:")
print("✅ Qdrant 생성 완료 (메모리 모드)")

✅ SentenceTransformer 모델 초기화 완료 (all-MiniLM-L6-v2)
✅ Qdrant 생성 완료 (메모리 모드)


# 컬렉션 생성

In [67]:
# ========================================
# 2. 컬렉션 생성
# ========================================
collection_name="test"

if qdrantClient.collection_exists(collection_name=collection_name):
    qdrantClient.delete_collection(collection_name=collection_name)
    print("✅ Qdrant 기존 컬렉션 삭제 완료")

qdrantClient.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": VectorParams(size=384, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams()
    }
)
print("✅ Qdrant 컬렉션 생성 완료")

✅ Qdrant 컬렉션 생성 완료


# 테스트 데이터 생성

In [68]:
# ========================================
# 3. 테스트 데이터 삽입
# =======================================
docs = [
    "미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진",
    "블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문",
    "강남 웨이팅 맛집, SNS에서 화제인 인기 브런치 카페",
    "전통 일식 오마카세, 제철 생선으로 만드는 스시 코스",
    "한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점"
]

# 테스트 데이터 임베딩 & BM25

In [70]:
# Dense 임베딩
dense_embeddings = sentenceTransformer.encode(docs)
print(f"✅ Dense 임베딩 생성: {dense_embeddings.shape}")

# BM25 인덱스 (전체 문서로 학습)
bm25 = BM25Okapi([doc.split() for doc in docs])
print(f"✅ BM25 인덱싱 완료")

✅ Dense 임베딩 생성: (5, 384)
✅ BM25 인덱싱 완료


# 테스트 데이터 저장 (Dense, Sparse)

In [71]:
# Qdrant에 삽입
points = []
for idx, (doc, dense) in enumerate(zip(docs, dense_embeddings)):
    sparse_scores = bm25.get_scores(doc.split())
    sparse_idx = np.where(sparse_scores > 0)[0]

    points.append(PointStruct(
        id=idx,
        vector={
            "dense": dense.tolist(),
            "sparse": SparseVector(
                indices=sparse_idx.tolist(),
                values=sparse_scores[sparse_idx].tolist(),
            )
        },
        payload={"text": doc, "shop_name": f"매장_{idx+1}"}
    ))

qdrantClient.upsert(collection_name=collection_name, points=points)
print(f"✅ {len(points)}개 포인트 삽입 완료")

✅ 5개 포인트 삽입 완료


# Hybrid Search (Dense + Sparse + RRF) 함수 정의

In [74]:
# ========================================
# 4. Hybrid Search 함수 (RRF)
# ========================================
def retrieve_similar_examples_hybrid(query, collection_name, top_k=2):
    # 1. query -> Dense 임베딩 생성
    query_dense_embedding = sentenceTransformer.encode(query)

    # 2. query -> Dense 검색
    dense_results = qdrantClient.query_points(
        collection_name=collection_name,
        query=query_dense_embedding.tolist(),
        using="dense",
        limit=10
    )

    # 3. query -> Sparse 벡터 생성 (BM25)
    query_tokens = query.split()
    query_sparse_scores = bm25.get_scores(query_tokens)
    query_sparse_idx = np.where(query_sparse_scores > 0)[0]

    # 4. query -> Sparse 검색
    sparse_results = qdrantClient.query_points(
        collection_name=collection_name,
        query=SparseVector(
            indices=sparse_idx.tolist(),
            values=sparse_scores[sparse_idx].tolist()
        ),
        using="sparse",
        limit=10
    )

    # 5. RFF 융합 (k=60은 표준 값)
    scores = defaultdict(float)
    k = 60

    # 6. Dense 점수 반영
    for rank, hit in enumerate(dense_results.points, 1):
        scores[hit.id] += 1 / (k + rank)

    # 7. Sparse 점수 반영
    for rank, hit in enumerate(sparse_results.points, 1):
        scores[hit.id] += 1 / (k + rank)

    # 8. 상위 top_k개 선택
    top_ids = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # 9. 실제 데이터 가져오기
    final_results = []
    for point_id, rrf_score in top_ids:
        point = qdrantClient.retrieve(collection_name=collection_name, ids=[point_id])[0]
        final_results.append({
            "shop_name": point.payload.get("shop_name", "Unknown"),
            "text": point.payload.get("text", ""),
            "score": rrf_score,
            "id": point_id
        })

    return final_results

# Hybrid Search (Dense + Sparse + RRF) 테스트

In [75]:
# ========================================
# 5. 검색 테스트
# ========================================

queries = [
   "프렌치 레스토랑",
   "스시 오마카세",
   "웨이팅 맛집"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"🔍 검색 쿼리: '{query}'")
    print(f"{'='*60}")

    results = retrieve_similar_examples_hybrid(query, collection_name, top_k=3)

    for idx, result in enumerate(results, 1):
        print(f"\n{idx}. {result['shop_name']} (Score: {result['score']:.4f})")
        print(f"   {result['text']}")


🔍 검색 쿼리: '프렌치 레스토랑'

1. 매장_2 (Score: 0.0325)
   블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문

2. 매장_5 (Score: 0.0325)
   한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점

3. 매장_1 (Score: 0.0159)
   미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진

🔍 검색 쿼리: '스시 오마카세'

1. 매장_5 (Score: 0.0323)
   한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점

2. 매장_2 (Score: 0.0323)
   블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문

3. 매장_4 (Score: 0.0164)
   전통 일식 오마카세, 제철 생선으로 만드는 스시 코스

🔍 검색 쿼리: '웨이팅 맛집'

1. 매장_5 (Score: 0.0323)
   한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점

2. 매장_2 (Score: 0.0315)
   블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문

3. 매장_1 (Score: 0.0164)
   미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진


# Dense vs Sparse vs Hybrid Search (RFF) 비교

In [80]:
# ========================================
# 6. Dense vs Sparse vs Hybrid 비교
# ========================================

print(f"\n\n{'='*60}")
print("📊 Dense vs Sparse vs Hybrid 비교")
print(f"{'='*60}")

test_queries = [
    "미쉐린",         # 정확한 키워드
    "고급 레스토랑",    # 의미적 검색
    "스시",           # 키워드
    "분위기 좋은 곳"    # 의미적
]

for query in test_queries:
    # Dense only
    dense_only = qdrantClient.query_points(
        collection_name=collection_name,
        query=sentenceTransformer.encode(query).tolist(),
        using="dense",
        limit=3
    )

    print(f"\n🔹 Dense Only:")
        
    for hit in dense_only.points:
        print(f"  - {hit.payload['text'][:50]}... (Score: {hit.score:.4f})")

    # Sparse only
    query_tokens = query.split()
    sparse_scores = bm25.get_scores(query_tokens)
    sparse_idx = np.where(sparse_scores > 0)[0]

    sparse_only = qdrantClient.query_points(
        collection_name=collection_name,
        query=SparseVector(
            indices=sparse_idx.tolist(),
            values=sparse_scores[sparse_idx].tolist()
        ),
        using="sparse",
        limit=3
    )

    print(f"\n🔹 Sparse Only:")
    for hit in sparse_only.points:
        print(f"  - {hit.payload['text'][:50]}... (Score: {hit.score:.4f})")

    # Hybrid (DemRRF)
    hybrid_results = retrieve_similar_examples_hybrid(query, collection_name, top_k=3)

    print(f"\n🔹 Hybrid (Dense + Sparse + RRF):")
    for result in hybrid_results:
        print(f"  - {result['text'][:50]}... (Score: {result['score']:.4f})")

print("\n✅ 테스트 완료!")



📊 Dense vs Sparse vs Hybrid 비교

🔹 Dense Only:
  - 전통 일식 오마카세, 제철 생선으로 만드는 스시 코스... (Score: 0.2541)
  - 미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진... (Score: 0.2464)
  - 강남 웨이팅 맛집, SNS에서 화제인 인기 브런치 카페... (Score: 0.2280)

🔹 Sparse Only:
  - 미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진... (Score: 9.2632)

🔹 Hybrid (Dense + Sparse + RRF):
  - 미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진... (Score: 0.0325)
  - 전통 일식 오마카세, 제철 생선으로 만드는 스시 코스... (Score: 0.0164)
  - 강남 웨이팅 맛집, SNS에서 화제인 인기 브런치 카페... (Score: 0.0159)

🔹 Dense Only:
  - 블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문... (Score: 0.7368)
  - 한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점... (Score: 0.6430)
  - 전통 일식 오마카세, 제철 생선으로 만드는 스시 코스... (Score: 0.5776)

🔹 Sparse Only:

🔹 Hybrid (Dense + Sparse + RRF):
  - 블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문... (Score: 0.0164)
  - 한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점... (Score: 0.0161)
  - 전통 일식 오마카세, 제철 생선으로 만드는 스시 코스... (Score: 0.0159)

🔹 Dense Only:
  - 강남 웨이팅 맛집, SNS에서 화제인 인기 브런치 카페... (Score: 0.3852)
  - 전통 일식 오마카세, 제철 생선으로 만드는 스시 코스... (Score: 0.3715)
  -